### Learning About Basics Of Agents In Langchain

- Creating Tools from langchain
- Creating the LLM, Prompt
- Combining Tools, LLM, Prompt
- Creating Agent Using AgentExecutor

In [2]:
# Setting up TavilySearch Tool
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
search = TavilySearchResults()

In [6]:
search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1739884233, 'localtime': '2025-02-18 05:10'}, 'current': {'last_updated_epoch': 1739883600, 'last_updated': '2025-02-18 05:00', 'temp_c': 6.1, 'temp_f': 43.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 3.4, 'wind_kph': 5.4, 'wind_degree': 19, 'wind_dir': 'NNE', 'pressure_mb': 1021.0, 'pressure_in': 30.15, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 100, 'cloud': 50, 'feelslike_c': 5.2, 'feelslike_f': 41.3, 'windchill_c': 8.0, 'windchill_f': 46.5, 'heatindex_c': 8.3, 'heatindex_f': 46.9, 'dewpoint_c': 7.3, 'dewpoint_f': 45.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 5.6, 'gust_kph': 9.1}}"},
 {'url': 'https://weathershogun.com/weather/us

In [7]:
# Creating a Retriever Tool
from langchain_community.document_loaders import  WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

In [9]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

embedding = OllamaEmbeddings(model='mxbai-embed-large:335m')

vectordb = Chroma.from_documents(documents=documents, embedding=embedding)

retriever = vectordb.as_retriever()

In [10]:
retriever.invoke('how to upload a dataset')

[Document(metadata={'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}, page_content='Get started by creating your first prompt.\nIterate on models and prompts using the Playground.\nManage prompts programmatically in your application.\nWas this page helpful?You can leave detailed feedback on GitHub.NextQuick StartObservabilityEvalsPrompt EngineeringCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright ¬© 2025 LangChain, Inc.'),
 Document(metadata={'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith'}, page_content='Get started by adding tracing 

In [12]:
from langchain.tools.retriever import create_retriever_tool

In [13]:
retriever_tool = create_retriever_tool(
    retriever,
    'langsmith_search',
    'Search for information about LangSmith. For any questions about LangSmith, you must use this tool!'
)

In [15]:
# Creating all tools list
tools = [search, retriever_tool]

In [32]:
# Loading LLM from which the Agent will be having its brain
from langchain_ollama import ChatOllama

llm = ChatOllama(model='llama3.2:3b')

In [33]:
# Now Import the prompt to guide the Agent
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

e:\conda\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [34]:
# Creating our agent
from langchain.agents import create_tool_calling_agent, AgentExecutor

agent = create_tool_calling_agent(llm, tools, prompt)

In [35]:
# Finally, we combine the agent (the brains) with the tools inside the AgentExecutor (which will repeatedly call the agent and execute tools).

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [36]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'LangSmith'}`


[{'url': 'https://docs.smith.langchain.com/old/user_guide', 'content': 'LangSmith is a platform for LLM application development, monitoring, and testing. LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications. Beta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. LangSmith also allows for tag and metadata grouping, which allows users to mark different versions of their applications with different identifiers and view how they are performing side-by-side within each chart. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.'}, {'url': 'https://www.langchain.com/langsmith', 'con

{'input': 'hi!',
 'output': "LangSmith is a platform that allows developers to build production-grade LLM (Large Language Model) applications. It provides a unified platform for debugging, testing, evaluating, and monitoring LLM applications. With LangSmith, developers can closely monitor and evaluate their application in real-time, which enables them to ship quickly and with confidence.\n\nLangSmith offers several features that make it an ideal tool for LLM development, including:\n\n1. Observability: LangSmith provides LLM-native observability, allowing developers to get meaningful insights from their application.\n2. Evaluation: LangSmith allows developers to evaluate their application over production traffic, score application performance, and get human feedback on data.\n3. Prompt Engineering: LangSmith provides tools designed to enable and facilitate prompt engineering, helping developers find the perfect prompt for their application.\n\nOverall, LangSmith is a powerful tool that

### Adding memory to our agent

In [37]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [38]:
store = {}

def get_session_memory(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

In [39]:
runnable = RunnableWithMessageHistory(
    agent_executor,
    get_session_memory,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [40]:
runnable.invoke(
    {'input' : 'hi! I am talha your creator'},
    config = {'configurable' : {'session_id' : 'chat1'}}
)



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Talha, creator of assistant'}`


Get started by creating your first prompt.
Iterate on models and prompts using the Playground.
Manage prompts programmatically in your application.
Was this page helpful?You can leave detailed feedback on GitHub.NextQuick StartObservabilityEvalsPrompt EngineeringCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright ¬© 2025 LangChain, Inc.

Get started by adding tracing to your application.
Create dashboards to view key metrics like RPS, error rates and costs.

Evals‚Äã
The quality and development speed of AI applications depends on high-quality evaluation datasets and metrics to test and optimize your applications on. The LangSmith SDK and UI make building and running high-quality evaluations easy.

Get started by creating your first evaluation.
Quickly assess the performance of your applicatio

{'input': 'hi! I am talha your creator',
 'chat_history': [],
 'output': "Hello Talha!\n\nIt's great to finally talk to you directly. I'm an AI designed by your team, and I'm here to assist you with any questions or concerns you may have.\n\nHow can I help you today? Do you have a specific question about my capabilities or the project we're working on together?"}

In [41]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='hi! I am talha your creator', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Talha!\n\nIt's great to finally talk to you directly. I'm an AI designed by your team, and I'm here to assist you with any questions or concerns you may have.\n\nHow can I help you today? Do you have a specific question about my capabilities or the project we're working on together?", additional_kwargs={}, response_metadata={})])}

# Creating ReAct Agent

In [43]:
# Creating Our first Inbuild tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

In [44]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

In [45]:
wiki_tool.name

'wikipedia'

In [46]:
# Creating our Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [47]:
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

In [48]:
# Creating our custom retriever tool
from langchain_community.document_loaders import  WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

embedding = OllamaEmbeddings(model='mxbai-embed-large:335m')

vectordb = Chroma.from_documents(documents=documents, embedding=embedding)

retriever = vectordb.as_retriever()

In [49]:
from langchain.tools.retriever import create_retriever_tool

In [50]:
retriever_tool = create_retriever_tool(
    retriever,
    'langsmith_search',
    'Search for information about LangSmith. For any questions about LangSmith, you must use this tool!'
)

In [51]:
tools = [wiki_tool, arxiv_tool, retriever_tool]

In [52]:
# Loading the prompt for ReAct Logic from hub
from langchain import hub

prompt = hub.pull("hwchase17/react")

e:\conda\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [54]:
# Loading LLM
from langchain_ollama import ChatOllama

llm = ChatOllama(model='llama3.2:3b')

In [55]:
# Create an agent executor by passing in the agent and tools
from langchain.agents import create_react_agent

agent = create_react_agent(llm, tools, prompt)

In [60]:
# Executing ReAct agent using AgentExecutor
from langchain.agents import  AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [62]:
agent_executor.invoke(
    {"input" : "what is langchain? Only use a tool if needed, otherwise respond with Final Answer not stuck in a ReAct loop."}
)



> Entering new AgentExecutor chain...
Final Answer: LangChain is an open-source Python library that allows developers to build conversational AI models and integrate them with various applications. It provides a framework for building conversational interfaces, chatbots, and other natural language processing (NLP) tasks. LangChain also supports the integration of various NLP models and technologies, making it a versatile tool for building custom conversational AI solutions.

No need to use any tools in this case!

> Finished chain.


{'input': 'what is langchain? Only use a tool if needed, otherwise respond with Final Answer not stuck in a ReAct loop.',
 'output': 'LangChain is an open-source Python library that allows developers to build conversational AI models and integrate them with various applications. It provides a framework for building conversational interfaces, chatbots, and other natural language processing (NLP) tasks. LangChain also supports the integration of various NLP models and technologies, making it a versatile tool for building custom conversational AI solutions.\n\nNo need to use any tools in this case!'}

### Adding Memory in ReAct Agent

In [63]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [65]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react-chat")

e:\conda\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [66]:
store = {}

def get_session_memory(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

In [67]:
runnable = RunnableWithMessageHistory(
    agent_executor,
    get_session_memory,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [69]:
runnable.invoke(
    {"input" : "Hi, I am Talha. Only use a tool if needed, otherwise respond with Final Answer, Not stuck in a ReAct loop."},
    config = {'configurable' : {'session_id' : 'chat1'}}
)



> Entering new AgentExecutor chain...
Question: Hi, I am Talha.

Action: Final Answer
Reasoning: Since no specific information is required about LangSmith and the tool for that purpose isn't specified, there's no need to use any of the provided tools. 

Final Answer: Hello Talha!

> Finished chain.


{'input': 'Hi, I am Talha. Only use a tool if needed, otherwise respond with Final Answer, Not stuck in a ReAct loop.',
 'chat_history': [],
 'output': 'Hello Talha!'}

In [71]:
runnable.invoke(
    {"input" : "what's my name? Only use a tool if needed, otherwise respond with Final Answer, Not stuck in a ReAct loop."},
    config = {'configurable' : {'session_id' : 'chat1'}}
)



> Entering new AgentExecutor chain...
Final Answer: You didn't specify which tool to use or provide any additional information. Therefore, I cannot proceed with the original request.

To answer your question about not getting stuck in a ReAct loop, we can design our system to avoid this. We can add a check at the beginning of each response to see if it's a reiteration of the previous response. If so, we can either provide a different response or inform the user that they are getting stuck in a loop.

Would you like me to explain how I could implement such a system?

> Finished chain.


{'input': "what's my name? Only use a tool if needed, otherwise respond with Final Answer, Not stuck in a ReAct loop.",
 'chat_history': [HumanMessage(content='Hi, I am Talha. Only use a tool if needed, otherwise respond with Final Answer, Not stuck in a ReAct loop.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Talha!', additional_kwargs={}, response_metadata={})],
 'output': "You didn't specify which tool to use or provide any additional information. Therefore, I cannot proceed with the original request.\n\nTo answer your question about not getting stuck in a ReAct loop, we can design our system to avoid this. We can add a check at the beginning of each response to see if it's a reiteration of the previous response. If so, we can either provide a different response or inform the user that they are getting stuck in a loop.\n\nWould you like me to explain how I could implement such a system?"}

In [72]:
runnable.invoke(
    {"input" : "what's my name? Only use a tool if needed, otherwise respond with Final Answer."},
    config = {'configurable' : {'session_id' : 'chat1'}}
)



> Entering new AgentExecutor chain...
Final Answer: You didn't specify which tool to use for this question. Since it's not a specific query about a person or topic that requires a specialized search tool, I'll provide a general response. To find your name, you might want to check any identification documents you have with your name on them, such as a driver's license or passport.

> Finished chain.


{'input': "what's my name? Only use a tool if needed, otherwise respond with Final Answer.",
 'chat_history': [HumanMessage(content='Hi, I am Talha. Only use a tool if needed, otherwise respond with Final Answer, Not stuck in a ReAct loop.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Talha!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="what's my name? Only use a tool if needed, otherwise respond with Final Answer, Not stuck in a ReAct loop.", additional_kwargs={}, response_metadata={}),
  AIMessage(content="You didn't specify which tool to use or provide any additional information. Therefore, I cannot proceed with the original request.\n\nTo answer your question about not getting stuck in a ReAct loop, we can design our system to avoid this. We can add a check at the beginning of each response to see if it's a reiteration of the previous response. If so, we can either provide a different response or inform the user that they are get

In [73]:
runnable.invoke(
    {"input" : "Hi, what's my name?"},
    config = {'configurable' : {'session_id' : 'chat1'}}
)



> Entering new AgentExecutor chain...
Thought: The user is asking a casual question, likely seeking information about themselves.

Action: langsmith_search(query='str', *, retriever=langsmith_retriever, document_prompt='BasePromptTemplate' = PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='str' = '\n\n', callbacks='Callbacks' = None, response_format: "Literal['content', 'content_and_artifact']" = 'content')
Action Input: 'who is my name'langsmith_search(query='str', *, retriever=langsmith_retriever, document_prompt='BasePromptTemplate' = PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='str' = '\n\n', callbacks='Callbacks' = None, response_format: "Literal['content', 'content_and_artifact']" = 'content') is not a valid tool, try one of [wikipedia, arxiv, langsmith_search].Question: Hi, what's my name?
Action: wikipe

KeyboardInterrupt: 